In [76]:
import numpy as np
import random
import itertools
from collections import defaultdict
#CONTENT CATALOGUE
#number of content items
K=50
#content relativity matrix
u_ij=np.zeros((K,K))
#percentage of cached items
C=0.2*K
C=int(C)
#cache
cache=[0]*C
#content relevance threshold
u_min=0.4
#USER MODEL
#number of recomendations
N=2
#probability of ending viewing session
q=0.02
#probability of picking one item with equal prob. in case all are relevant
alpha=0.9
#probability when the user chooses from K items
p_k=1/K

actions=[]
actions_initialized=0



#state action space

def nested_dict():
    return defaultdict(nested_dict)
Space=nested_dict()


#def initialize_cache():
  #initialize cache
 # for i in range(0,C):
  #    cache[i]=i+30

def initialize_cache():
   items_in_cache = random.sample(list(range(K)), C)
   for i in range(len(items_in_cache)):
    cache[i]=items_in_cache[i]


def initialize_relativity_matrix():
  #content relativity matrix initializations
  for i in range(0,K):
    for j in range(0,i):
      u_ij[i,j]=np.random.rand()

def item_cost(item):
  if item in cache:
    return 10
  else:
    return 0

#check if all items are above u_min
def all_bigger_than_u_min(current_item,recommended_items):
  for i in recommended_items:
    relevance=u_ij[i,current_item]
    if(relevance<u_min):
      return False
  return True



def initialize_state_action_space():
  for state in range(K):
    #print(state)
    items = list(range(K))
    other_items = items.copy()
    #other_items.remove(state)
    recommended_combinations = list(itertools.combinations(other_items, N))

    for item_set in recommended_combinations:
          item_set_str = [str(item) for item in item_set]  # Convert integers to strings
          action_name = '-'.join(item_set_str)
          action_outcomes_list=[]
          global actions_initialized
          if actions_initialized == 0:
            actions.append(action_name)


          if(state in item_set):
            for i in other_items:
              action_outcomes_list.append((0,0,0,False))
          elif all_bigger_than_u_min(state,item_set):
            #print(action_name)
            #print(state)
            prob_of_picking_recommended=(1/N)*alpha*(1-q)
            prob_of_picking_random=p_k*(1-alpha)*(1-q)
            for i in other_items:
              if i in item_set:
                action_outcomes_list.append((prob_of_picking_recommended,i,item_cost(i),False))
              else:
                action_outcomes_list.append((prob_of_picking_random,i,item_cost(i),False))

            #print(state)
          else:
            for i in other_items:
              prob_of_picking_random=p_k*(1-q)
              action_outcomes_list.append((prob_of_picking_random,i,item_cost(i),False))

          action_outcomes_list.append((q,0,0,True))#with some probability end session
          Space[state][action_name]=action_outcomes_list
  actions_initialized=1
initialize_relativity_matrix()
initialize_cache()
  #print(cache)
initialize_state_action_space()
print(Space[6])


defaultdict(<function nested_dict at 0x7f7ada9a6200>, {'0-1': [(0.0196, 0, 0, False), (0.0196, 1, 0, False), (0.0196, 2, 0, False), (0.0196, 3, 0, False), (0.0196, 4, 0, False), (0.0196, 5, 0, False), (0.0196, 6, 10, False), (0.0196, 7, 10, False), (0.0196, 8, 0, False), (0.0196, 9, 0, False), (0.0196, 10, 0, False), (0.0196, 11, 10, False), (0.0196, 12, 0, False), (0.0196, 13, 0, False), (0.0196, 14, 0, False), (0.0196, 15, 0, False), (0.0196, 16, 0, False), (0.0196, 17, 0, False), (0.0196, 18, 10, False), (0.0196, 19, 0, False), (0.0196, 20, 0, False), (0.0196, 21, 0, False), (0.0196, 22, 0, False), (0.0196, 23, 0, False), (0.0196, 24, 0, False), (0.0196, 25, 0, False), (0.0196, 26, 0, False), (0.0196, 27, 0, False), (0.0196, 28, 0, False), (0.0196, 29, 10, False), (0.0196, 30, 10, False), (0.0196, 31, 0, False), (0.0196, 32, 10, False), (0.0196, 33, 10, False), (0.0196, 34, 0, False), (0.0196, 35, 0, False), (0.0196, 36, 0, False), (0.0196, 37, 10, False), (0.0196, 38, 0, False), (0

In [77]:
import tkinter as tk #loads standard python GUI libraries
import numpy as np
import time
import random
import array


P=Space.copy()

# The next few lines are mostly for accounting
Tmax = 100000
size = len(P)


t = 0




#this one is generic to be applied in many AI gym compliant environments

def policy_evaluation(pi, P, gamma = 1.0, epsilon = 1e-10):  #inputs: (1) policy to be evaluated, (2) model of the environment (transition probabilities, etc., see previous cell), (3) discount factor (with default = 1), (4) convergence error (default = 10^{-10})
    t = 0   #there's more elegant ways to do this
    prev_V = np.zeros(len(P)) # use as "cost-to-go", i.e. for V(s')
    #print(len(P[0]))
    while True:
        V = np.zeros(len(P)) # current value function to be learnerd
        for s in range(len(P)):  # do for every state
            for prob, next_state, reward, done in P[s][pi(s)]:  # calculate one Bellman step --> i.e., sum over all probabilities of transitions and reward for that state, the action suggested by the (fixed) policy, the reward earned (dictated by the model), and the cost-to-go from the next state (which is also decided by the model)
                V[s] += prob * (reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < epsilon: #check if the new V estimate is close enough to the previous one;
            break # if yes, finish loop
        prev_V = V.copy() #freeze the new values (to be used as the next V(s'))
        t += 1
    #print(len(P[0]))
    return V

def policy_improvement(V, P, gamma=1.0):  # takes a value function (as the cost to go V(s')), a model, and a discount parameter
    Q = np.zeros((len(P), len(P[0])), dtype=np.float64) #create a Q value array\
    action_index=[]
    for s in range(len(P)):        # for every state in the environment/model
        action_counter=0
        for a in list(P[0].keys()):  # and for every action in that state
            action_index.append(a)
            for prob, next_state, reward, done in P[s][a]:  #evaluate the action value based on the model and Value function given (which corresponds to the previous policy that we are trying to improve)
                Q[s][action_counter] += prob * (reward + gamma * V[next_state] * (not done))
            action_counter+=1
    new_pi_dict = {}
    Q_argmax = np.argmax(Q, axis=1)

    for s, a in enumerate(Q_argmax):
        new_pi_dict[s] = action_index[a]
        #print(action_index[a])

    new_pi = lambda s: new_pi_dict[s]  # this basically creates the new (improved) policy by choosing at each state s the action a that has the highest Q value (based on the Q array we just calculated)
    return new_pi

# policy iteration is simple, it will call alternatively policy evaluation then policy improvement, till the policy converges.

def policy_iteration(P, gamma = 1.0, epsilon = 1e-10):
    t = 0
    random_actions = np.random.choice(tuple(P[0].keys()), len(P))     # start with random actions for each state
    pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]     # and define your initial policy pi_0 based on these action (remember, we are passing policies around as python "functions", hence the need for this second line)

    while True:
        P_opt_dict = {s: pi(s) for s in range(K)}
        for key in P_opt_dict:
            item = P_opt_dict[key]
            print(f"Key: {key}, Item: {item}")
        old_pi = {s: pi(s) for s in range(len(P))}  #keep the old policy to compare with new
        V = policy_evaluation(pi,P,gamma,epsilon)   #evaluate latest policy --> you receive its converged value function        print(V)
        print(V)
        pi = policy_improvement(V,P,gamma)          #get a better policy using the value function of the previous one just calculated
        t += 1
        if old_pi == {s:pi(s) for s in range(len(P))}: # you have converged to the optimal policy if the "improved" policy is exactly the same as in the previous step
            break
    print('converged after %d iterations' %t) #keep track of the number of (outer) iterations to converge
    return V,pi





V_opt,P_opt = policy_iteration(P,0.5,0.001)   #just example of calling the various new functions we created.


print(V_opt)
P_opt_dict = {s: P_opt(s) for s in range(len(V_opt))}
for key in P_opt_dict:
    item = P_opt_dict[key]
    print(f"Key: {key}, Item: {item}")


Key: 0, Item: 16-39
Key: 1, Item: 17-36
Key: 2, Item: 32-39
Key: 3, Item: 33-47
Key: 4, Item: 36-45
Key: 5, Item: 21-35
Key: 6, Item: 18-39
Key: 7, Item: 6-15
Key: 8, Item: 2-32
Key: 9, Item: 14-31
Key: 10, Item: 7-30
Key: 11, Item: 37-48
Key: 12, Item: 0-4
Key: 13, Item: 17-26
Key: 14, Item: 22-43
Key: 15, Item: 1-4
Key: 16, Item: 9-48
Key: 17, Item: 1-33
Key: 18, Item: 3-23
Key: 19, Item: 37-39
Key: 20, Item: 15-35
Key: 21, Item: 34-47
Key: 22, Item: 14-44
Key: 23, Item: 6-20
Key: 24, Item: 18-23
Key: 25, Item: 33-39
Key: 26, Item: 18-46
Key: 27, Item: 21-41
Key: 28, Item: 5-37
Key: 29, Item: 15-33
Key: 30, Item: 8-45
Key: 31, Item: 13-14
Key: 32, Item: 11-45
Key: 33, Item: 13-17
Key: 34, Item: 27-32
Key: 35, Item: 18-46
Key: 36, Item: 29-42
Key: 37, Item: 22-25
Key: 38, Item: 3-34
Key: 39, Item: 30-41
Key: 40, Item: 5-31
Key: 41, Item: 15-48
Key: 42, Item: 27-28
Key: 43, Item: 41-44
Key: 44, Item: 39-46
Key: 45, Item: 6-23
Key: 46, Item: 18-33
Key: 47, Item: 18-25
Key: 48, Item: 5-1